In [4]:
import cv2
import numpy as np
import glob


In [ ]:
def detect_robot(image):

    height, width, _ = image.shape
    lower_quarter = image[4 * height // 5 :, :] #crop to the lower quarter (to ignore ceiling lights)

    # Convert to grayscale
    gray = cv2.cvtColor(lower_quarter, cv2.COLOR_BGR2GRAY)

    # Enhance brightness (of the robot light)
    bright = cv2.convertScaleAbs(gray, alpha=1.5, beta=0)

    # brightness thresholding
    brightness_threshold = 40 #TODO better threshold?
    _, mask = cv2.threshold(bright, brightness_threshold, 255, cv2.THRESH_BINARY)

    mask = cv2.bitwise_not(mask) #reversed mask to detect dark



    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on the lower half of the image
    #cv2.drawContours(lower_half, contours, -1, (0, 255, 0), 2)  # Green contours
    #print(len(contours))


    # Filter contours by length
    robot_detected = False
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > width // 9:  # Line is larger than 1/4 of the image width at which point we should dodge the robot TODO better strategy?
            robot_detected = True
            # Show detected line
            cv2.rectangle(lower_quarter, (x, y), (x + w, y + h), (0, 0, 255), 2)


    # Combine the lower quarter back with the upper half for display
    output_image = np.vstack((image[: height // 5 * 4, :], lower_quarter))

    return output_image, robot_detected, mask


In [1]:
def detect_robot(image):

    height, width, _ = image.shape
    lower_quarter = image[3 * height // 4 :, :] #crop to the lower quarter (to ignore ceiling lights)

    # Convert to grayscale
    gray = cv2.cvtColor(lower_quarter, cv2.COLOR_BGR2GRAY)

    # Enhance brightness (of the robot light)
    bright = cv2.convertScaleAbs(gray, alpha=1.5, beta=0)

    # brightness thresholding
    brightness_threshold = 40 #TODO better threshold?
    _, mask = cv2.threshold(bright, brightness_threshold, 255, cv2.THRESH_BINARY)

    mask = cv2.bitwise_not(mask) #reversed mask to detect dark



    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw contours on the lower half of the image
    #cv2.drawContours(lower_half, contours, -1, (0, 255, 0), 2)  # Green contours
    #print(len(contours))


    # Filter contours by length
    robot_detected = False
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > width // 9:  # Line is larger than 1/4 of the image width at which point we should dodge the robot TODO better strategy?
            robot_detected = True
            # Show detected line
            cv2.rectangle(lower_quarter, (x, y), (x + w, y + h), (0, 0, 255), 2)


    # Combine the lower quarter back with the upper half for display
    output_image = np.vstack((image[: height // 4 * 3, :], lower_quarter))

    return output_image, robot_detected, mask


In [63]:
#image = cv2.imread('images_obstacle/image_20241125_145857.jpg')
image = cv2.imread('images_obstacle/image_20241125_145833.jpg')
img_detected, robot_detected, img_mask = detect_robot(image)
print('Robot in image: ', robot_detected)

cv2.imwrite('processed_image.jpg', img_detected)
cv2.imwrite('mask_image.jpg', img_mask)


Robot in image:  True


True

In [64]:
image_files = glob.glob("images_obstacle/*.jpg")

for i, filename in enumerate(image_files):
    print(filename)
    img = cv2.imread(filename)
    img_detected, robot_detected, img_mask = detect_robot(img)

    cv2.imwrite(f'{filename}_detection.jpg', img_detected)
    cv2.imwrite(f'{filename}_mask.jpg', img_mask)


images_obstacle/image_20241125_150017.jpg
images_obstacle/image_20241125_150014.jpg
images_obstacle/image_20241125_145916.jpg
images_obstacle/image_20241125_145926.jpg
images_obstacle/image_20241125_150053.jpg
images_obstacle/image_20241125_150051.jpg
images_obstacle/image_20241125_150055.jpg
images_obstacle/image_20241125_145937.jpg
images_obstacle/image_20241125_145923.jpg
images_obstacle/image_20241125_145857.jpg
images_obstacle/image_20241125_145936.jpg
images_obstacle/image_20241125_145950.jpg
images_obstacle/image_20241125_145833.jpg
images_obstacle/image_20241125_145947.jpg
images_obstacle/image_20241125_145943.jpg
images_obstacle/image_20241125_145942.jpg
images_obstacle/image_20241125_150034.jpg


In [83]:
def detect_robot_blackness(image):
    height, width, _ = image.shape
    lower_quarter = image[7* height // 8 :, :] #crop to the lower quarter (to ignore ceiling lights)

    # Convert to grayscale
    gray = cv2.cvtColor(lower_quarter, cv2.COLOR_BGR2GRAY)

    # Enhance brightness (of the robot light)
    bright = cv2.convertScaleAbs(gray, alpha=1.5, beta=0)

    # brightness thresholding
    brightness_threshold = 48 #TODO better threshold?
    _, mask = cv2.threshold(bright, brightness_threshold, 255, cv2.THRESH_BINARY)



    # proportion black pixels
    total_pixels = mask.size  # Total number of pixels
    white_pixels = cv2.countNonZero(mask)  # non-zero (black) pixels
    #print(mask)
    black_pixels = total_pixels-white_pixels
    black_proportion = black_pixels / total_pixels
    print(black_proportion)

    # Determine if the robot is detected
    robot_detected = black_proportion > 0.18  # True if more than 18% of the mask is black

    # Visualize the result
    if robot_detected:
        cv2.putText(
            lower_quarter,
            f"Robot Detected: {black_proportion*100:.2f}% black",
            (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX,
            1,
            (0, 0, 255),
            2,
        )

    # Combine the lower quarter back with the upper part for display
    output_image = np.vstack((image[: height // 8 * 7, :], lower_quarter))

    return output_image, robot_detected, mask

In [81]:
"""Single image"""
image = cv2.imread('images_obstacle/image_20241125_145937.jpg') #should start to dodge: 150017, 145857, 145916 (extremely close); not: 145937
img_detected, robot_detected, img_mask = detect_robot_blackness(image)
print('Robot in image: ', robot_detected)

cv2.imwrite('processed_image.jpg', img_detected)
cv2.imwrite('mask_image.jpg', img_mask)

0.1343984375
Robot in image:  False


True

In [84]:
image_files = glob.glob("images_obstacle/*.jpg")

for i, filename in enumerate(image_files):
    print(filename)
    img = cv2.imread(filename)
    img_detected, robot_detected, img_mask = detect_robot_blackness(img)

    cv2.imwrite(f'{filename}_detection.jpg', img_detected)
    cv2.imwrite(f'{filename}_mask.jpg', img_mask)


images_obstacle/image_20241125_150017.jpg
0.2087734375
images_obstacle/image_20241125_150014.jpg
0.0832421875
images_obstacle/image_20241125_145916.jpg
0.2608359375
images_obstacle/image_20241125_145926.jpg
0.103625
images_obstacle/image_20241125_150053.jpg
0.0593359375
images_obstacle/image_20241125_150051.jpg
0.052953125
images_obstacle/image_20241125_150055.jpg
0.0524375
images_obstacle/image_20241125_145937.jpg
0.1343984375
images_obstacle/image_20241125_145923.jpg
0.123015625
images_obstacle/image_20241125_145857.jpg
0.112796875
images_obstacle/image_20241125_145936.jpg
0.351171875
images_obstacle/image_20241125_145950.jpg
0.055625
images_obstacle/image_20241125_145833.jpg
0.039046875
images_obstacle/image_20241125_145947.jpg
0.0490625
images_obstacle/image_20241125_145943.jpg
0.1574609375
images_obstacle/image_20241125_150034.jpg
0.0175546875
